# Combines results across chromosomes

Post-run program to merge tables (summary and expanded table) across chromosomes to enable genome-wide analysis

Instructions:
1. Run each cell from the beginning in order
2. Modify the variables where suggested

In [1]:
import pandas as pd
import os
import glob
import re

In [2]:
def _chrom_order(chrom):
    """
    Map chr names to an integer order
    """
    m = re.match(r'^chr(\d+)$', chrom)
    if m:
        return int(m.group(1))
    if chrom in ('chrX', 'X'):
        return 23
    if chrom in ('chrY', 'Y'):
        return 24
    if chrom in ('chrM', 'chrMT', 'MT'):
        return 25
    # put everything else at the end
    return float('inf')

In [8]:
def extract_comments(csv_path: str) -> list[str]:
    """
    Read the leading comments “#” from a csv and return them 
    Stops as soon as it hits a non-# line
    """
    comments = []
    with open(csv_path, 'r') as fh:
        for line in fh:
            if line.startswith('#'):
                # strip newline, then replace chr<number> with chr*
                txt = line.rstrip('\n')
                txt = re.sub(r'chr\d+', 'chr*', txt)
                comments.append(txt)
            else:
                break
    return comments

# Repli-HiC_K562_WT_totalS_chr_combinedKb

def combine_results(results_dir, folder_pattern, result_type, exp_name=None):
    """
    Search for folders under `results_dir` matching `folder_pattern` (which may
    contain a glob-style asterisk), then for each folder:
      - load <folder>/<folder>_results_<result_type>/<folder>_summary_table.csv
      - load <folder>/<folder>_results_<result_type>/<folder>_expanded_table.csv
    and concatenate all summaries and all expanded tables into two DataFrames.

    Parameters
    ----------
    results_dir : str
        Path to the directory containing all result‐folders
    folder_pattern : str
        A glob style pattern (e.g. "splenic-B-cell_*_50Kb") used to match subfolders
    result_type : str
        The suffix after "_results" in each folder (e.g. "all", "p-0.01", etc)

    Returns
    -------
    (combined_summary, combined_expanded) : tuple of pd.DataFrame
        The concatenated summary and expanded tables
    """
    # build the full glob pattern
    search_pattern = os.path.join(results_dir, folder_pattern)

    prefix, suffix = folder_pattern.split('*')

    print(prefix, suffix)

    if exp_name is not None:
        exp_prefix, exp_suffix = exp_name.split('*')
        folder_regex = re.compile(
            rf"^{re.escape(prefix)}(.+?){re.escape(suffix)}$"
        )

    # If this notebook is already run, then do not include the (already combined) combined folder
    exclude_name = f"{prefix}_combined{suffix}"
    print(exclude_name)

    pattern = re.compile(
        rf"^{re.escape(prefix)}"      # “Repli-HiC_K562_WT_totalS_chr”
        r".+" # Changed to match any character because ce10 genome is roman numerals
        rf"{re.escape(suffix)}$"      # “_50Kb”
    )

    matched_folders = [
        os.path.join(results_dir, d)
        for d in os.listdir(results_dir)
        if pattern.match(d)
    ]

    print(matched_folders)

    # Exclude the combined folder if it exists
    matched_folders = [
        f for f in matched_folders
        if os.path.basename(f) != exclude_name
    ]

    if not matched_folders:
        raise FileNotFoundError(f"No folders found matching {search_pattern}")

    summary_frames = []
    expanded_frames = []
    bedpe_frames = []
    for i, folder in enumerate(matched_folders):

        if exp_name is not None:
            m = folder_regex.match(os.path.basename(folder))
            if not m:
                raise ValueError(
                    f"Folder name '{os.path.basename(folder)}' "
                    f"did not match pattern '{folder_pattern}'"
                )
            wild = m.group(1)
            base = f"{exp_prefix}{wild}{exp_suffix}"
        else:
            # basename is the actual folder name without the path
            base = os.path.basename(folder)

        # path to the results‐type subdirectory
        res_subdir = os.path.join(folder, f"{base}_results_{result_type}")

        # csv paths
        summary_csv = os.path.join(res_subdir, f"{base}_summary_table.csv")
        expanded_csv = os.path.join(res_subdir, f"{base}_expanded_table.csv")
        juicer_bedpe = os.path.join(res_subdir, f"{base}_juicer-visualize.bedpe")

        # Load and collect
        if not os.path.isfile(summary_csv):
            raise FileNotFoundError(f"Expected file not found: {summary_csv}")
        if not os.path.isfile(expanded_csv):
            raise FileNotFoundError(f"Expected file not found: {expanded_csv}")
        if not os.path.isfile(juicer_bedpe):
            raise FileNotFoundError(f"Expected file not found: {juicer_bedpe}")
        
        # extract the comments from a single summary csv only
        if i == 0:
            comments = extract_comments(summary_csv)
            for comment in comments:
                print(comment)

        summary_frames.append(pd.read_csv(summary_csv, comment='#'))
        expanded_frames.append(pd.read_csv(expanded_csv, comment='#'))
        bedpe_frames.append(pd.read_csv(juicer_bedpe, comment='#', sep='\t', header=None, index_col=False, 
                                        names=['chrom1', 'start1', 'end1', 'chrom2', 'start2', 'end2']))

    # Concatenate all and reset the index
    combined_summary = pd.concat(summary_frames,  ignore_index=True)
    combined_expanded = pd.concat(expanded_frames, ignore_index=True)
    combined_bedpe = pd.concat(bedpe_frames, ignore_index=True)

    # Sort 
    # First convert chromosomes to a numerical value for sorting
    combined_summary["chrom_order"] = combined_summary["chrom"].apply(_chrom_order)
    combined_expanded["chrom_order"] = combined_expanded["chrom"].apply(_chrom_order)
    combined_bedpe["chrom1_order"] = combined_bedpe["chrom1"].apply(_chrom_order)
    combined_bedpe["chrom2_order"] = combined_bedpe["chrom2"].apply(_chrom_order)
    # Then simply sort by chromosome order and position
    combined_summary.sort_values(by=["chrom_order"], inplace=True)
    combined_expanded.sort_values(by=["chrom_order"], inplace=True)
    combined_bedpe.sort_values(by=["chrom1_order", "start1", "chrom2_order", "start2"], inplace=True)
    # Drop the temporary chrom_order column 
    combined_summary.drop(columns=["chrom_order"], inplace=True)
    combined_expanded.drop(columns=["chrom_order"], inplace=True)
    combined_bedpe.drop(columns=["chrom1_order", "chrom2_order"], inplace=True)

    # Reset index 
    combined_summary.reset_index(drop=True, inplace=True)
    combined_expanded.reset_index(drop=True, inplace=True)
    combined_bedpe.reset_index(drop=True, inplace=True)

    return combined_summary, combined_expanded, combined_bedpe, comments

# Save the combined tables
def save_csv(df, save_dir, parameter_str_comment):
    """
    Save a dataframe to a csv file 

    Parameters
    ----------
    df : pd.DataFrame
        The input dataframe to save
    save_dir : str
        The name of the file to save the dataframe to
    parameter_str : list
        A list containing parameters to be included in the beginning of the csv file as comments
        Each element in the list is a string that must start with a special character (e.g '#')
    
    Returns
    -------
    None, but saves the dataframe to a csv file
    """
    with open(save_dir, "w") as f:
        f.write("\n".join(parameter_str_comment) + "\n")
        df.to_csv(f, index=False)

# Individual

**MODIFY the following cell as needed**

In [4]:
# Let the notebook know what experiment type you would like to combine
# Specify it as a folder pattern with an asterisk * to match multiple directories
# Example: "splenic-B-cell_WT_insitu-hic_GSE82144_mm9_chr*_50Kb" will match all chromosomes
# of that specific experiment run
# folder_pattern = "splenic-B-cell_WT_insitu-hic_Kieffer-Kwon-2018_GSE82144_mm9_chr*_50Kb"
# folder_pattern = "DP-thymocytes_WT_hic_Guo-2022_GSE199059_mm10-remapped_chr*_50Kb"
# folder_pattern = "GSE199059_CD69negDPWTR1R2R3R4_merged_chr*_50Kb"
# folder_pattern = "Repli-HiC_K562_WT_totalS_chr*_50Kb"
# folder_pattern = "GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr*_50Kb"
# folder_pattern = "GSE199059_CD69negDPWTR1R2R3R4_merged-normalized-2_chr*_50Kb"
# folder_pattern = "GSE199059_CD69negDPWTR1R2R3R4_merged-root-2_chr*_50Kb"
# folder_pattern = "Repli-HiC_K562_WT_totalS-stripiness-1_chr*_50Kb"
folder_pattern = "GSE130275_mESC_WT_combined_2.6B_chr*_50Kb"

exp_name = None
# exp_name = "GSE199059_CD69negDPWTR1R2R3R4_merged_chr*_50Kb" 
# exp_name = "Repli-HiC_K562_WT_totalS_chr*_50Kb"

# Location of the results folders
# Where is the folder above located?
results_dir = "/nfs/turbo/umms-minjilab/sionkim/miajet_output/"

# This is the suffix after the "_results" in each folder 
# You can specify "all" to combine all results or a specifically thresholded output
# Examples: "all", "p-0.01", "saliency-90", "saliency-90-p-0.1"
result_type = ["saliency-90-p-0.1", "saliency-90-p-0.05", "saliency-90-p-0.01", "saliency-90", "p-0.1", "p-0.05", "p-0.01", "all"]

In [9]:
if isinstance(result_type, str):
    df_s, df_e, df_b, parameter_str = combine_results(results_dir, folder_pattern, result_type)
    # Will print out the parametesr of the experiment
    # So that you can confirm this is indeed the correct experiment you are combining

    df_summary = [df_s]
    df_expanded = [df_e]
    df_bedpe = [df_b]
else:
    df_summary = []
    df_expanded = []
    df_bedpe = []
    for rt in result_type:
        df_s, df_e, df_b, parameter_str = combine_results(results_dir, folder_pattern, rt, exp_name=exp_name)
        df_summary.append(df_s)
        df_expanded.append(df_e)
        df_bedpe.append(df_b)

GSE130275_mESC_WT_combined_2.6B_chr _50Kb
GSE130275_mESC_WT_combined_2.6B_chr_combined_50Kb
['/nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE130275_mESC_WT_combined_2.6B_chr_combined_50Kb']


FileNotFoundError: No folders found matching /nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE130275_mESC_WT_combined_2.6B_chr*_50Kb

In [6]:
# Can visually inspect the summary table
df_summary[0]

,unique_id,chrom,start,end,length,input_mean,angle_mean,width_mean,jet_saliency,ks,p-val_raw,p-val_corr,stripiness
0,chr1_28_9,chr1,1.739396e+08,1.743253e+08,900000,0.132,110.422,7.109,0.054,1.000,0.000,0.000,0.041
1,chr1_64_10,chr1,7.121808e+07,7.159345e+07,1000000,0.121,109.471,4.348,0.051,0.450,0.017,0.054,0.025
2,chr1_1541_10,chr1,1.739067e+08,1.742917e+08,800000,0.134,113.905,7.393,0.041,0.938,0.000,0.000,0.035
3,chr1_419_11,chr1,1.020671e+08,1.026250e+08,850000,0.080,88.292,7.144,0.047,1.000,0.000,0.000,0.014
4,chr1_20_16,chr1,1.857944e+08,1.867769e+08,1250000,0.132,78.523,8.578,0.075,0.760,0.000,0.000,0.039
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,chrX_105_14,chrX,6.600922e+07,6.685632e+07,3200000,0.012,119.451,9.802,0.008,0.219,0.047,0.079,0.002
158,chrX_1044_4,chrX,1.712626e+07,1.737590e+07,400000,0.012,90.720,2.755,0.006,1.000,0.000,0.001,0.002
159,chrX_4016_4,chrX,1.063709e+08,1.066141e+08,450000,0.015,89.417,3.999,0.006,0.667,0.017,0.042,0.003
160,chrX_119_9,chrX,3.953824e+07,3.976459e+07,400000,0.012,98.998,4.649,0.005,0.875,0.001,0.008,0.001


In [7]:
# Can visually inspect the expanded table
df_expanded[0]

,unique_id,chrom,x (bp),y (bp),x (pixels),y (pixels),width,angle_imagej,ridge_strength
0,chr1_20_16,chr1,1.865853e+08,1.848068e+08,5252.691,58.993,9.163,90.723,0.010
1,chr1_20_16,chr1,1.866257e+08,1.847748e+08,5252.810,57.970,9.119,91.995,0.010
2,chr1_20_16,chr1,1.866675e+08,1.847436e+08,5252.961,56.937,8.761,93.373,0.009
3,chr1_20_16,chr1,1.867117e+08,1.847136e+08,5253.161,55.888,8.042,94.894,0.009
4,chr1_20_16,chr1,1.867377e+08,1.846681e+08,5252.885,54.877,8.277,96.592,0.008
...,...,...,...,...,...,...,...,...,...
2033,chrX_13_8,chrX,1.886369e+07,1.848282e+07,528.574,78.759,3.567,90.397,0.001
2034,chrX_13_8,chrX,1.889771e+07,1.844674e+07,528.545,77.768,3.567,89.579,0.001
2035,chrX_105_14,chrX,6.681408e+07,6.434742e+07,1855.318,49.262,11.877,125.818,0.001
2036,chrX_105_14,chrX,6.668986e+07,6.484107e+07,1860.542,58.000,11.941,117.658,0.001


In [8]:
# Can visually inspect the bedpe table
df_bedpe[0]

,chrom1,start1,end1,chrom2,start2,end2
0,chr1,43036676,43061676,chr1,44117732,44142732
1,chr1,43074360,43099360,chr1,44084735,44109735
2,chr1,43108486,43133486,chr1,44048054,44073054
3,chr1,43140552,43165552,chr1,44010325,44035325
4,chr1,43158386,43183386,chr1,43967960,43992960
...,...,...,...,...,...,...
2033,chrX,155950983,155975983,chrX,156334898,156359898
2034,chrX,155992574,156017574,chrX,156307032,156332032
2035,chrX,156033195,156058195,chrX,156277527,156302527
2036,chrX,156072731,156097731,chrX,156246313,156271313


**(OPTIONAL) modify as needed**

In [9]:
save_name = folder_pattern.replace('*', f'_combined') 
# This is the save name for the combined results

save_dir = os.path.join(results_dir, save_name)
# Folder to save the combined results (summary and expanded tables)
# You may overwrite the `results_dir` if you wish to save to a different location

print("Saving combined tables to:")
print(f"* Folder location: {save_dir}")

save_dir_summary = []
save_dir_expanded = []
save_dir_bedpe = []
for rt in result_type:
    print(f"* Result type: {rt}")
    save_name_file = folder_pattern.replace('*', f'_combined_{rt}')

    # Below is the code to print out the save locations to help you modify the variables abovef
    # Once satisfied, move on to the next cell
    save_dir_s = os.path.join(save_dir, f"{save_name_file}_summary_table.csv")
    save_dir_e = os.path.join(save_dir, f"{save_name_file}_expanded_table.csv")
    save_dir_b = os.path.join(save_dir, f"{save_name_file}_juicer-visualize.bedpe")

    save_dir_summary.append(save_dir_s)
    save_dir_expanded.append(save_dir_e)
    save_dir_bedpe.append(save_dir_b)

    print(f"\t* Summary table: {save_name_file}_summary_table.csv")
    print(f"\t* Expanded table: {save_name_file}_expanded_table.csv")
    print(f"\t* Bedpe table: {save_name_file}_juicer-visualize.bedpe")

Saving combined tables to:
* Folder location: /nfs/turbo/umms-minjilab/sionkim/miajet_output/splenic-B-cell_WT_insitu-hic_Kieffer-Kwon-2018_GSE82144_mm9_chr_combined_50Kb
* Result type: saliency-90-p-0.1
	* Summary table: splenic-B-cell_WT_insitu-hic_Kieffer-Kwon-2018_GSE82144_mm9_chr_combined_saliency-90-p-0.1_50Kb_summary_table.csv
	* Expanded table: splenic-B-cell_WT_insitu-hic_Kieffer-Kwon-2018_GSE82144_mm9_chr_combined_saliency-90-p-0.1_50Kb_expanded_table.csv
	* Bedpe table: splenic-B-cell_WT_insitu-hic_Kieffer-Kwon-2018_GSE82144_mm9_chr_combined_saliency-90-p-0.1_50Kb_juicer-visualize.bedpe
* Result type: saliency-90-p-0.05
	* Summary table: splenic-B-cell_WT_insitu-hic_Kieffer-Kwon-2018_GSE82144_mm9_chr_combined_saliency-90-p-0.05_50Kb_summary_table.csv
	* Expanded table: splenic-B-cell_WT_insitu-hic_Kieffer-Kwon-2018_GSE82144_mm9_chr_combined_saliency-90-p-0.05_50Kb_expanded_table.csv
	* Bedpe table: splenic-B-cell_WT_insitu-hic_Kieffer-Kwon-2018_GSE82144_mm9_chr_combined_sali

In [10]:
# Make directory if it doesn't exist already and save the files
os.makedirs(save_dir, exist_ok=True)

for i in range(len(result_type)):
    save_csv(df_summary[i], save_dir_summary[i], parameter_str)
    save_csv(df_expanded[i], save_dir_expanded[i], parameter_str)
    df_bedpe[i].to_csv(save_dir_bedpe[i], sep='\t', index=False, header=False) 
    print(f"Saved\n *{save_dir_summary[i]}\n *{save_dir_expanded[i]}\n *{save_dir_bedpe[i]}")

Saved
 */nfs/turbo/umms-minjilab/sionkim/miajet_output/splenic-B-cell_WT_insitu-hic_Kieffer-Kwon-2018_GSE82144_mm9_chr_combined_50Kb/splenic-B-cell_WT_insitu-hic_Kieffer-Kwon-2018_GSE82144_mm9_chr_combined_saliency-90-p-0.1_50Kb_summary_table.csv
 */nfs/turbo/umms-minjilab/sionkim/miajet_output/splenic-B-cell_WT_insitu-hic_Kieffer-Kwon-2018_GSE82144_mm9_chr_combined_50Kb/splenic-B-cell_WT_insitu-hic_Kieffer-Kwon-2018_GSE82144_mm9_chr_combined_saliency-90-p-0.1_50Kb_expanded_table.csv
 */nfs/turbo/umms-minjilab/sionkim/miajet_output/splenic-B-cell_WT_insitu-hic_Kieffer-Kwon-2018_GSE82144_mm9_chr_combined_50Kb/splenic-B-cell_WT_insitu-hic_Kieffer-Kwon-2018_GSE82144_mm9_chr_combined_saliency-90-p-0.1_50Kb_juicer-visualize.bedpe
Saved
 */nfs/turbo/umms-minjilab/sionkim/miajet_output/splenic-B-cell_WT_insitu-hic_Kieffer-Kwon-2018_GSE82144_mm9_chr_combined_50Kb/splenic-B-cell_WT_insitu-hic_Kieffer-Kwon-2018_GSE82144_mm9_chr_combined_saliency-90-p-0.05_50Kb_summary_table.csv
 */nfs/turbo/umms

# Multiple samples


Loads in the `submit_all_config.yaml` used to submit batch jobs on Slurm to now combine the results

In [11]:
import yaml

import sys
sys.path.append('..')
from utils.plotting import genomic_labels

In [12]:
with open("../submit_all_config.yaml") as cf:
    config = yaml.safe_load(cf)

samples = config["samples"]

results_dir = "/nfs/turbo/umms-minjilab/sionkim/miajet_output/"
result_types = [
    "saliency-90-p-0.1", "saliency-90-p-0.05", "saliency-90-p-0.01",
    "saliency-90", "p-0.1", "p-0.05", "p-0.01", "all"
]

for key, attributes in samples.items():
    print(f"Processing experiment: {key}")

    f = attributes["file"]
    basename = os.path.basename(f)
    exp = os.path.splitext(basename)[0]  # Remove file extension

    resolution = genomic_labels(int(attributes["res"]))

    folder_pattern = f"{exp}_chr*_{resolution}"
    save_name = folder_pattern.replace('*', '_combined')
    save_dir = os.path.join(results_dir, save_name)
    os.makedirs(save_dir, exist_ok=True)

    # Combine results for each result type
    df_summary_list = []
    df_expanded_list = []
    df_bedpe_list   = []
    for rt in result_types:
        print(f"\tCombining result type: {rt}")
        df_s, df_e, df_b, parameter_str = combine_results(results_dir, folder_pattern, rt)
        df_summary_list.append(df_s)
        df_expanded_list.append(df_e)
        df_bedpe_list.append(df_b)

        # Construct file paths
        save_base = folder_pattern.replace('*', f"_combined_{rt}")
        summary_fp = os.path.join(save_dir, f"{save_base}_summary_table.csv")
        expanded_fp = os.path.join(save_dir, f"{save_base}_expanded_table.csv")
        bedpe_fp = os.path.join(save_dir, f"{save_base}_juicer-visualize.bedpe")

        # Save outputs
        save_csv(df_s, summary_fp, parameter_str)
        save_csv(df_e, expanded_fp, parameter_str)
        df_b.to_csv(bedpe_fp, sep='\t', index=False, header=False)

        print(f"\tSaved summary:  {summary_fp}")
        print(f"\tSaved expanded: {expanded_fp}")
        print(f"\tSaved BEDPE:    {bedpe_fp}")

    print()  


Processing experiment: Repli-HiC_K562_WT_totalS
	Combining result type: saliency-90-p-0.1
Repli-HiC_K562_WT_totalS_chr_combined_50Kb
# Repli-HiC_K562_WT_totalS_chr*_50Kb
# Inputs
# * Hi-C file (.hic): /nfs/turbo/umms-minjilab/downloaded_data/Repli-HiC_K562_WT_totalS.hic
# 
# Required Parameters
# * Experiment type: replihic
# * Chromosome: chr*
# * Resolution: 50000
# * Save directory root: /nfs/turbo/umms-minjilab/sionkim/miajet_output
# 
# Extended Parameters
# * Significance threshold(s): [0.1, 0.05, 0.01]
# * Window size: 6000000
# * Normalization: VC_SQRT
# * Data type: observed
# * Jet widths (if specified): None
# * Angle range: [80, 100]
# * Saliency threshold on zero removed: 90
# * Hysteresis thresholding parameters: [0.01, 0.05]
# * Root within: None
# * Folder name: Repli-HiC_K562_WT_totalS_chr*_50Kb
# * Save directory: /nfs/turbo/umms-minjilab/sionkim/miajet_output/Repli-HiC_K562_WT_totalS_chr*_50Kb
# * Number of cores: 4
# * Verbose: True
# 
# Optional Parameters
# * Scal

/tmp/ipykernel_1691840/1278568343.py:124: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_summary = pd.concat(summary_frames,  ignore_index=True)
/tmp/ipykernel_1691840/1278568343.py:125: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_expanded = pd.concat(expanded_frames, ignore_index=True)


	Saved summary:  /nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged_chr_combined_50Kb/GSE199059_CD69negDPWTR1R2R3R4_merged_chr_combined_saliency-90-p-0.01_50Kb_summary_table.csv
	Saved expanded: /nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged_chr_combined_50Kb/GSE199059_CD69negDPWTR1R2R3R4_merged_chr_combined_saliency-90-p-0.01_50Kb_expanded_table.csv
	Saved BEDPE:    /nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged_chr_combined_50Kb/GSE199059_CD69negDPWTR1R2R3R4_merged_chr_combined_saliency-90-p-0.01_50Kb_juicer-visualize.bedpe
	Combining result type: saliency-90
GSE199059_CD69negDPWTR1R2R3R4_merged_chr_combined_50Kb
# GSE199059_CD69negDPWTR1R2R3R4_merged_chr*_50Kb
# Inputs
# * Hi-C file (.hic): /nfs/turbo/umms-minjilab/downloaded_data/GSE199059_CD69negDPWTR1R2R3R4_merged.hic
# 
# Required Parameters
# * Experiment type: hic
# * Chromosome: chr*
# * Resolution: 50000
# * Save direct

/tmp/ipykernel_1691840/1278568343.py:124: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_summary = pd.concat(summary_frames,  ignore_index=True)
/tmp/ipykernel_1691840/1278568343.py:125: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_expanded = pd.concat(expanded_frames, ignore_index=True)


	Saved summary:  /nfs/turbo/umms-minjilab/sionkim/miajet_output/GM12878_RNAPII_chiadrop_Kim-2024_4DNFI3ZH8UYR_hg38_chr_combined_50Kb/GM12878_RNAPII_chiadrop_Kim-2024_4DNFI3ZH8UYR_hg38_chr_combined_saliency-90-p-0.05_50Kb_summary_table.csv
	Saved expanded: /nfs/turbo/umms-minjilab/sionkim/miajet_output/GM12878_RNAPII_chiadrop_Kim-2024_4DNFI3ZH8UYR_hg38_chr_combined_50Kb/GM12878_RNAPII_chiadrop_Kim-2024_4DNFI3ZH8UYR_hg38_chr_combined_saliency-90-p-0.05_50Kb_expanded_table.csv
	Saved BEDPE:    /nfs/turbo/umms-minjilab/sionkim/miajet_output/GM12878_RNAPII_chiadrop_Kim-2024_4DNFI3ZH8UYR_hg38_chr_combined_50Kb/GM12878_RNAPII_chiadrop_Kim-2024_4DNFI3ZH8UYR_hg38_chr_combined_saliency-90-p-0.05_50Kb_juicer-visualize.bedpe
	Combining result type: saliency-90-p-0.01
GM12878_RNAPII_chiadrop_Kim-2024_4DNFI3ZH8UYR_hg38_chr_combined_50Kb
# GM12878_RNAPII_chiadrop_Kim-2024_4DNFI3ZH8UYR_hg38_chr*_50Kb
# Inputs
# * Hi-C file (.hic): /nfs/turbo/umms-minjilab/downloaded_data/GM12878_RNAPII_chiadrop_Kim-20

...

/tmp/ipykernel_1691840/1278568343.py:124: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_summary = pd.concat(summary_frames,  ignore_index=True)
/tmp/ipykernel_1691840/1278568343.py:125: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_expanded = pd.concat(expanded_frames, ignore_index=True)


	Saved summary:  /nfs/turbo/umms-minjilab/sionkim/miajet_output/mESC_CTCF-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10_chr_combined_50Kb/mESC_CTCF-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10_chr_combined_saliency-90-p-0.01_50Kb_summary_table.csv
	Saved expanded: /nfs/turbo/umms-minjilab/sionkim/miajet_output/mESC_CTCF-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10_chr_combined_50Kb/mESC_CTCF-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10_chr_combined_saliency-90-p-0.01_50Kb_expanded_table.csv
	Saved BEDPE:    /nfs/turbo/umms-minjilab/sionkim/miajet_output/mESC_CTCF-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10_chr_combined_50Kb/mESC_CTCF-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10_chr_combined_saliency-90-p-0.01_50Kb_juicer-visualize.bedpe
	Combining result type: saliency-90
mESC_CTCF-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10_chr_combined_50Kb
# mESC_CTCF-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10_chr*_50Kb
# Inputs
# * Hi-C file (.hic): /nfs/turbo/umms-minjilab/downloaded_data/mESC_CTCF-auxin-3hr_m

/tmp/ipykernel_1691840/1278568343.py:124: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_summary = pd.concat(summary_frames,  ignore_index=True)
/tmp/ipykernel_1691840/1278568343.py:125: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_expanded = pd.concat(expanded_frames, ignore_index=True)


	Saved summary:  /nfs/turbo/umms-minjilab/sionkim/miajet_output/mESC_RAD21-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10_chr_combined_50Kb/mESC_RAD21-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10_chr_combined_saliency-90-p-0.01_50Kb_summary_table.csv
	Saved expanded: /nfs/turbo/umms-minjilab/sionkim/miajet_output/mESC_RAD21-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10_chr_combined_50Kb/mESC_RAD21-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10_chr_combined_saliency-90-p-0.01_50Kb_expanded_table.csv
	Saved BEDPE:    /nfs/turbo/umms-minjilab/sionkim/miajet_output/mESC_RAD21-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10_chr_combined_50Kb/mESC_RAD21-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10_chr_combined_saliency-90-p-0.01_50Kb_juicer-visualize.bedpe
	Combining result type: saliency-90
mESC_RAD21-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10_chr_combined_50Kb
# mESC_RAD21-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10_chr*_50Kb
# Inputs
# * Hi-C file (.hic): /nfs/turbo/umms-minjilab/downloaded_data/mESC_RAD21-au

FileNotFoundError: No folders found matching /nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE130275_mESC_WT_combined_2.6B_chr*_50Kb